In [ ]:
## imports
import numpy as np
from numpy import pi
import seaborn as sns
from IPython.display import Image
import matplotlib.pyplot as plt

from scipy.optimize import minimize 
from scipy.interpolate import griddata
import math
from typing import Union

In [ ]:
## qiskit import
from qiskit import *
from qiskit.algorithms import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import quantum_info, IBMQ, Aer
from qiskit import BasicAer, transpile
from qiskit.utils import QuantumInstance
from qiskit.quantum_info import Statevector, partial_trace , DensityMatrix
from qiskit.circuit import Parameter, ParameterExpression, ParameterVector
# backend = BasicAer.get_backend("statevector_simulator")
# quantum_instance = QuantumInstance(backend)
from qiskit.algorithms import optimizers, AmplitudeEstimation, EstimationProblem, AmplificationProblem, Grover, GroverResult, AmplitudeAmplifier
from qiskit.circuit.library import LinearAmplitudeFunction, LinearPauliRotations, PiecewiseLinearPauliRotations, WeightedAdder, GroverOperator
from qiskit_finance.circuit.library import LogNormalDistribution, NormalDistribution
#from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.visualization import plot_histogram, plot_state_qsphere, plot_bloch_multivector, plot_bloch_vector


In [ ]:
## IBM setup ~
# IBMQ.save_account("1c62e8a0d2b058d0e797db9c811bc34582df5553e25812301cd1518662c0ab68d2378ac6c197b65f8be944d04b7e7439f034e3926a44fa8088538b3d13782c1a", overwrite= True)
provider = IBMQ.load_account()
# IBMQ.get_provider(hub='ibm-q-education', group='iit-madras-1', project='quantum-computin')
# setup required backends 
lima = provider.get_backend('ibmq_lima')
# construct circuit for uncertainty modeler.get_backend('ibmq_lima')
manila = provider.get_backend('ibmq_manila')
qsm = Aer.get_backend('qasm_simulator')
stv = Aer.get_backend('statevector_simulator')
aer = Aer.get_backend('aer_simulator')

In [ ]:
## helper fucntions ~
def str_to_oracle(pattern: str, name= 'oracle', return_type = "QuantumCircuit" ) -> Union[QuantumCircuit,  Statevector] :
    """ Convert a given string to an oracle circuit
        ARGS:
             pattern: a numpy vector with binarry entries 
        RETURNS: 
                QuantumCircuit   """

    l = len(pattern)
    qr = QuantumRegister(l, name='reg')
    a = AncillaRegister(1, name='ancilla')
    oracle_circuit = QuantumCircuit(qr, a, name= name+str(pattern))
    for q in range(l):
        if(pattern[q]=='0'): oracle_circuit.x(qr[q])
    oracle_circuit.x(a)
    oracle_circuit.h(a)
    oracle_circuit.mcx(qr, a)
    oracle_circuit.h(a)
    oracle_circuit.x(a)
    for q in range(l):
        if(pattern[q]=='0'): oracle_circuit.x(qr[q])
    
    #oracle_circuit.barrier()
    if return_type == "QuantumCircuit":
        return oracle_circuit

## oracle prep ~
def generate_oracles(good_states: list) -> QuantumCircuit :
    """ Return a QuantumCircuit that implements the oracles given the good_states
        ARGS:
            good_states: list of good staes, states must be binary strings, Eg. ['00', '11']
        RETURNS:
            QuantumCircuit """

    oracles = [ str_to_oracle(good_state) for good_state in good_states ]
    oracle_circuit = oracles[0]
    for oracle in oracles[1:] :
        oracle_circuit.compose(oracle,  inplace= True)
    
    return oracle_circuit

## state prep circuit ~
def state_prep_circuit(n):
    qreg = QuantumRegister(n)
    state_prep_circuit = QuantumCircuit(qreg, name = "state_prep")
    state_prep_circuit.h(qreg)

    return state_prep_circuit

def plot_histogram_cmpr(counts, figsize= (15,8)):
    return plot_histogram([counts[0], counts[1]], figsize= figsize, legend= ['grover', 'variational'], title= 'Comparison')


In [ ]:
## optimisation step ~
def objective(var_params , state_prep_circuit , var_prep_circuit,  grover_circuit, grover_k = 1 , shots= 1024) :
    """ function to compare the performace of the varriational circuit with the grover circuit 
        ARGS: 
            var_params: variational parameters to be used in the variational circuit 
            state_prep_circuit: fucntion to generate circuit for preapring the initial state
            var_preps_circuit: function to generate the variational circuit 
            grover_circuit: circuit that implements the grover operator 
            grover_k: power to which the grover operator is to be raised, is implemented as iterative composition of the 'grover_circuit'
        
        RETURNS: 
                required fidelity between the grover circuit and the variational circuit """
    
    num_state_qubits = grover_circuit.num_qubits - grover_circuit.num_ancillas
    qc = state_prep_circuit(num_state_qubits)
    state_prep_circuit_var = qc.copy()
    state_prep_circuit_grover = qc.copy()

    ## var ~
    var_circuit = var_prep_circuit(num_state_qubits, var_params_input= var_params )
    state_prep_circuit_var.compose( var_circuit, inplace= True )
    
    ## grover ~
    num_ancillas=  grover_circuit.num_ancillas
    ancilla = AncillaRegister(num_ancillas, name= 'ancilla')
    state_prep_circuit_grover.add_register(ancilla)
    for iter in range(grover_k):
            state_prep_circuit_grover.compose( grover_circuit, inplace= True)

    ## measure fidelity using 'Statevector' method ~
    var_statevector = Statevector.from_instruction(state_prep_circuit_var)
    # print(partial_trace(Statevector.from_instruction(state_prep_circuit_grover), [num_state_qubits] )) ##checkflag
    grover_statevector = partial_trace(Statevector.from_instruction(state_prep_circuit_grover), [num_state_qubits] ).to_statevector()
    
    fidelity = -1*np.real(var_statevector.inner(grover_statevector))
    print('fidelity :', fidelity) ##checkflag

    return fidelity


def test_var_vs_grover(var_params , state_prep_circuit , var_prep_circuit,  grover_circuit, grover_k= 1, shots= 1024, plot= True):
    """ function to compare the performace of the varriational circuit with the grover circuit 
        ARGS: 
            var_params: variational parameters to be used in the variational circuit 
            state_prep_circuit: fucntion to generate circuit for preapring the initial state
            var_preps_circuit: function to generate the variational circuit 
            grover_circuit: circuit that implements the grover operator 
            grover_k: power to which the grover operator is to be raised, is implemented as iterative composition of the 'grover_circuit'
        
        RETURNS: 
            [grover_counts, var_counts] : a list containing the 'counts' dictionary obtained by executing the grover circuit and the variational circuit.
            state_prep_circuit_grover : circuit for grover 
            state_prep_circuit_var :  variational circuit
            
            """
    
    num_state_qubits = grover_circuit.num_qubits - grover_circuit.num_ancillas
    creg = ClassicalRegister(num_state_qubits, name= 'creg' )

    qc = state_prep_circuit(num_state_qubits)
    state_prep_circuit_var = qc.copy()
    state_prep_circuit_grover = qc.copy()

    ## var ~
    var_circuit = var_prep_circuit(num_state_qubits, var_params_input= var_params, insert_barriers= True )
    state_prep_circuit_var.compose( var_circuit, inplace= True )
    state_prep_circuit_var.measure_all()

    var_counts = execute(state_prep_circuit_var, backend= qsm, shots= 1024).result().get_counts()

    ## grover ~
    num_ancillas=  grover_circuit.num_ancillas
    qreg_grover = QuantumRegister(num_state_qubits, name= 'qreg')
    state_prep_circuit_grover = QuantumCircuit(qreg_grover).compose(state_prep_circuit_grover)
    ancilla = AncillaRegister(num_ancillas, name= 'ancilla')
    creg = ClassicalRegister(num_state_qubits, name= 'creg')
    state_prep_circuit_grover.add_register(ancilla)
    state_prep_circuit_grover.add_register(creg)
    for iter in range(grover_k):
        state_prep_circuit_grover.compose( grover_circuit, inplace= True)

    state_prep_circuit_grover.measure(qreg_grover, creg )
    grover_counts = execute(state_prep_circuit_grover, backend= qsm, shots= shots).result().get_counts()

    ## plot histogram ~
    # if plot == True:
        

    return [grover_counts, var_counts] , state_prep_circuit_grover, state_prep_circuit_var 


## Variational Quantum Amplitude Estimation Algorithm 
Here we try to implement QAE based on variational quantum circuits as discussed in the paper <insert link >. 

### Step:
Designning a variational circuit to implement $\mathcal{Q}^k$ on a given initial state, where $\mathcal{Q}$ is the grover operator composed as 
$ \mathcal{Q} = - (\mathbb{I} - 2 P_{\psi_0})(\mathbb{I} - 2 P_{good})$. The variational circuit will be constructed out of paramterised $R_y$ gates and $CX$ gates, as discussed in the original paper. 

For implementation we will use pre-defined functions such as  ```AmplificationProblem```, ```Grover``` , ```GroverOperator``` from ```qiskit.circuit.library``` and ```qiskit.algorithms``` module.

##### #2 qubit states

In [ ]:
## variational circuit preparation ~
def var_circuit_2qubit(n=2 , var_params_input= 'random' , num_var_params= 4):
    """ Prepare a variational circuit that will optimised to emulate the action of a Grover circuit 
        ARGS: 
            n: no. of qubits [**Prefixed]
            var_params_input: variational parmaters to be used in the circuit. If 'random' the parmaters are intiialised randomly
        RETURNS:
            QuantumCircuit 
    """
        
    qreg = QuantumRegister(n, name= 'qreg')
    var_circuit = QuantumCircuit(qreg, name= "Var_"+str(n))
    
    if var_params_input == 'random' : var_params = np.random.uniform(low= 0, high= 2*pi, size= num_var_params)
    else: var_params = var_params_input
    
    var_circuit.ry(var_params[0], qreg[1] )
    var_circuit.cx(1,0)
    var_circuit.ry(var_params[1], qreg[1])
    var_circuit.ry(var_params[2], qreg[0])
    var_circuit.cx(1,0)
    var_circuit.ry(var_params[3], qreg[1])

    return var_circuit

In [ ]:
## init ~
good_states = ['01']
n = len(good_states[0])
initial_state = None

## create grover operator ~
oracle_circuit = generate_oracles(good_states)
grover_operator = GroverOperator(oracle_circuit, name= "Grover", insert_barriers= False)

In [ ]:
num_var_params = 4

## scipy optimisation ~
optimisation_result = minimize(objective, np.random.uniform(low= 0, high= 2*pi, size= num_var_params), (state_prep_circuit, var_circuit_2qubit, grover_operator), 'SLSQP')
print('params: ', optimisation_result.x)

## qiskit-based optimisation ~
# adam = optimizers.ADAM(maxiter=500000)
# optimisation_result = adam.minimize(objective, np.random.uniform(low= 0, high= 2*pi, size= num_var_params) )


In [ ]:
result = test_var_vs_grover(optimisation_result.x, state_prep_circuit, var_circuit_2qubit, grover_operator)
# plot_histogram(result[0])
result[1].draw(), result[2].draw()

#### #4qubit states

In [ ]:
## variational circuit preparation ~
def var_circuit_4qubit(n=4 , var_params_input= 'random' , num_var_layers= 2, insert_barriers= False):
    """ Prepare a variational circuit that will optimised to emulate the action of a Grover circuit 
        ARGS: 
            n: no. of qubits [**Prefixed]
            var_params_input: variational parmaters to be used in the circuit. If 'random' the parmaters are intiialised randomly
            var_layers: no. of times a single variational unit is appended against itself. N.B every variational unit has same structure 

        RETURNS:
            QuantumCircuit 
    """
        
    qreg = QuantumRegister(n, name= 'qreg')
    var_circuit = QuantumCircuit(qreg, name= "Var_"+str(n))
    num_var_params = 10
    
    if var_params_input == 'random' : var_params = np.random.uniform(low= 0, high= 2*pi, size= num_var_params * var_layers)
    else: var_params = var_params_input
    
    for layer in range(0, num_var_layers):
        # print('layer :', layer) ##checkflag
        if insert_barriers== True: var_circuit.barrier()
        var_circuit.ry(var_params[num_var_params*layer + 0], 1)
        var_circuit.ry(var_params[num_var_params*layer + 1], 3)
  

        var_circuit.cx(1,0)
        var_circuit.ry(var_params[num_var_params*layer + 2], 0)
        var_circuit.ry(var_params[num_var_params*layer + 3], 1)
        var_circuit.cx(1,0)
        var_circuit.cx(3,2)
        var_circuit.ry(var_params[num_var_params*layer + 4], 2)
        var_circuit.ry(var_params[num_var_params*layer + 5], 3)
        var_circuit.cx(3,2)

        var_circuit.ry(var_params[num_var_params*layer + 6], 1)
        var_circuit.cx(1,2)
        var_circuit.ry(var_params[num_var_params*layer + 7], 2)
        var_circuit.cx(2,3)
        var_circuit.ry(var_params[num_var_params*layer + 8], 3)
        var_circuit.cx(3,0)
        var_circuit.ry(var_params[num_var_params*layer + 9], 0)
        var_circuit.cx(0,1)

    return var_circuit

In [ ]:
## init ~
good_states = ['1101', '0010']
n = len(good_states[0])
initial_state = None

## create grover operator ~
oracle_circuit = generate_oracles(good_states)
grover_operator_4qubit = GroverOperator(oracle_circuit, name= "Grover", insert_barriers= False)

In [ ]:
var_layers = 2
num_var_params = 10 * var_layers
optimisation_result_4qubit = minimize(objective, np.random.uniform(low= 0, high= 2*pi, size= num_var_params), (state_prep_circuit, var_circuit_4qubit, grover_operator_4qubit), 'COBYLA')
optimisation_result_4qubit = minimize(objective, optimisation_result_4qubit.x, (state_prep_circuit, var_circuit_4qubit, grover_operator_4qubit), 'BFGS')
optimisation_result_4qubit = minimize(objective, optimisation_result_4qubit.x, (state_prep_circuit, var_circuit_4qubit, grover_operator_4qubit), 'COBYLA')
# optimisation_result_4qubit = minimize(objective, optimisation_result_4qubit.x, (state_prep_circuit, var_circuit_6qubit, grover_operator_4qubit), 'COBYLA')
print('params: ', optimisation_result_4qubit.x)

In [ ]:
r = test_var_vs_grover(optimisation_result_4qubit.x, state_prep_circuit, var_circuit_4qubit, grover_operator_4qubit)

In [ ]:
r[2].draw()

In [ ]:
plot_histogram_cmpr(r[0])

#### #6 qubit states

In [ ]:
## variational circuit preparation ~
def var_circuit_6qubit(n=6 , var_params_input= 'random' , num_var_layers= 3, insert_barriers= False):
    """ Prepare a variational circuit that will optimised to emulate the action of a Grover circuit 
        ARGS: 
            n: no. of qubits [**Prefixed]
            var_params_input: variational parmaters to be used in the circuit. If 'random' the parmaters are intiialised randomly
            var_layers: no. of times a single variational unit is appended against itself. N.B every variational unit has same structure 

        RETURNS:
            QuantumCircuit 
    """
        
    qreg = QuantumRegister(n, name= 'qreg')
    var_circuit = QuantumCircuit(qreg, name= "Var_"+str(n))
    num_var_params = 15 
    
    if var_params_input == 'random' : var_params = np.random.uniform(low= 0, high= 2*pi, size= num_var_params * var_layers)
    else: var_params = var_params_input
    
    for layer in range(0, num_var_layers):
        # print('layer :', layer) ##checkflag
        if insert_barriers== True: var_circuit.barrier()
        var_circuit.ry(var_params[num_var_params*layer + 0], 1)
        var_circuit.ry(var_params[num_var_params*layer + 1], 3)
        var_circuit.ry(var_params[num_var_params*layer + 2], 5)

        var_circuit.cx(1,0)
        var_circuit.ry(var_params[num_var_params*layer + 3], 0)
        var_circuit.ry(var_params[num_var_params*layer + 4], 1)
        var_circuit.cx(1,0)
        var_circuit.cx(3,2)
        var_circuit.ry(var_params[num_var_params*layer + 5], 2)
        var_circuit.ry(var_params[num_var_params*layer + 6], 3)
        var_circuit.cx(3,2)
        var_circuit.cx(5,4)
        var_circuit.ry(var_params[num_var_params*layer + 7], 4)
        var_circuit.ry(var_params[num_var_params*layer + 8], 5)
        var_circuit.cx(5,4)

        var_circuit.ry(var_params[num_var_params*layer + 9], 2)
        var_circuit.ry(var_params[num_var_params*layer + 10], 4)

        var_circuit.cx(2,1)
        var_circuit.ry(var_params[num_var_params*layer + 11], 1)
        var_circuit.ry(var_params[num_var_params*layer + 12], 2)
        var_circuit.cx(2,1)
        var_circuit.cx(4,3)
        var_circuit.ry(var_params[num_var_params*layer + 13], 3)
        var_circuit.ry(var_params[num_var_params*layer + 14], 4)
        var_circuit.cx(4,3)

    return var_circuit

In [ ]:
## init ~
good_states = ['011001', '001100']
n = len(good_states[0])
initial_state = None

## create grover operator ~
oracle_circuit = generate_oracles(good_states)
grover_operator = GroverOperator(oracle_circuit, name= "Grover", insert_barriers= False)

In [ ]:
var_layers = 3
num_var_params = 15 * var_layers
# optimisation_result = minimize(objective, np.random.uniform(low= 0, high= 2*pi, size= num_var_params), (state_prep_circuit, var_circuit_6qubit, grover_operator), 'COBYLA')
optimisation_result = minimize(objective, optimisation_result.x, (state_prep_circuit, var_circuit_6qubit, grover_operator), 'BFGS')
optimisation_result = minimize(objective, optimisation_result.x, (state_prep_circuit, var_circuit_6qubit, grover_operator), 'COBYLA')
# optimisation_result = minimize(objective, optimisation_result.x, (state_prep_circuit, var_circuit_6qubit, grover_operator), 'COBYLA')
print('params: ', optimisation_result.x)

In [ ]:
optimal_params_6qubit = optimisation_result.x
r = test_var_vs_grover(optimisation_result.x, state_prep_circuit, var_circuit_6qubit, grover_operator)
plot_histogram_cmpr(r[0], figsize= (25,15))

In [ ]:
r[2].draw()

##### Alternate optimsation for 6 qubit case

In [ ]:
def objective_6qubit():
    